# Direct Marketing with Amazon SageMaker Autopilot
---

---

## Contents

1. [Introduction](#Introduction)
1. [Prerequisites](#Prerequisites)
1. [Downloading the dataset](#Downloading)
1. [Upload the dataset to Amazon S3](#Uploading)
1. [Setting up the SageMaker Autopilot Job](#Settingup)
1. [Launching the SageMaker Autopilot Job](#Launching)
1. [Tracking Sagemaker Autopilot Job Progress](#Tracking)
1. [Results](#Results)
1. [Cleanup](#Cleanup)

## Introduction

Amazon SageMaker Autopilot is an automated machine learning (commonly referred to as AutoML) solution for tabular datasets. You can use SageMaker Autopilot in different ways: on autopilot (hence the name) or with human guidance, without code through SageMaker Studio, or using the AWS SDKs. This notebook, as a first glimpse, will use the AWS SDKs to simply create and deploy a machine learning model.

A typical introductory task in machine learning (the "Hello World" equivalent) is one that uses a dataset to predict whether a customer will enroll for a term deposit at a bank, after one or more phone calls. For more information about the task and the dataset used, see [Bank Marketing Data Set](https://archive.ics.uci.edu/ml/datasets/bank+marketing).

Direct marketing, through mail, email, phone, etc., is a common tactic to acquire customers.  Because resources and a customer's attention are limited, the goal is to only target the subset of prospects who are likely to engage with a specific offer.  Predicting those potential customers based on readily available information like demographics, past interactions, and environmental factors is a common machine learning problem. You can imagine that this task would readily translate to marketing lead prioritization in your own organization.

This notebook demonstrates how you can use Autopilot on this dataset to get the most accurate ML pipeline through exploring a number of potential options, or "candidates". Each candidate generated by Autopilot consists of two steps. The first step performs automated feature engineering on the dataset and the second step trains and tunes an algorithm to produce a model. When you deploy this model, it follows similar steps. Feature engineering followed by inference, to decide whether the lead is worth pursuing or not. The notebook contains instructions on how to train the model as well as to deploy the model to perform batch predictions on a set of leads. Where it is possible, use the Amazon SageMaker Python SDK, a high level SDK, to simplify the way you interact with Amazon SageMaker.

Other examples demonstrate how to customize models in various ways. For instance, models deployed to devices typically have memory constraints that need to be satisfied as well as accuracy. Other use cases have real-time deployment requirements and latency constraints. For now, keep it simple.

## Prerequisites

Before you start the tasks in this tutorial, do the following:

- The Amazon Simple Storage Service (Amazon S3) bucket and prefix that you want to use for training and model data. This should be within the same Region as Amazon SageMaker training. The code below will create, or if it exists, use, the default bucket.
- The IAM role to give Autopilot access to your data. See the Amazon SageMaker documentation for more information on IAM roles: https://docs.aws.amazon.com/sagemaker/latest/dg/security-iam.html

In [1]:
%mkdir -p ~/tmp/autopilot

In [2]:
%cd ~/tmp/autopilot/

/Users/vbalasubramaniam/tmp/autopilot


In [3]:
%env AWS_PROFILE=trifacta_master_trial      
# This profile must have permissions to invoke Sagemaker
# Also update trust relationship to allow the service principal sagemaker.amazonaws.com to assume your role

env: AWS_PROFILE=trifacta_master_trial


In [4]:
import sagemaker
import boto3
from sagemaker import get_execution_role

region = boto3.Session().region_name

session = sagemaker.Session()
bucket = session.default_bucket()
prefix = 'sagemaker/autopilot-dm'

role = get_execution_role()

sm = boto3.Session().client(service_name='sagemaker',region_name=region)

In [5]:
import boto3       # Make sure to use a recent version of boto3 that supports automl
boto3.__version__  # If you have an older version, do `pip install boto3 --upgrade` and restart your kernel

'1.12.5'

## Downloading the dataset<a name="Downloading"></a>
Download the [direct marketing dataset](!wget -N https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip) from the sample data s3 bucket. 

\[Moro et al., 2014\] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

In [6]:
#!wget -N https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
#!unzip -o bank-additional.zip

#local_data_path = './bank-additional/bank-additional-full.csv'


In [7]:
trifacta_bucket = 'trifacta-partnerdemo-trifactabucket-kkcpnw234feu'
trifacta_key = 'trifacta/queryResults/admin@trifacta.local/bank-additional-full_2.csv'
local_data_path = 'bank-additional-full.csv'
s3 = boto3.Session().client(service_name='s3',region_name=region)
s3.download_file(Bucket=trifacta_bucket, Key=trifacta_key, Filename=local_data_path)

## Upload the dataset to Amazon S3<a name="Uploading"></a>

Before you run Autopilot on the dataset, first perform a check of the dataset to make sure that it has no obvious errors. The Autopilot process can take long time, and it's generally a good practice to inspect the dataset before you start a job. This particular dataset is small, so you can inspect it in the notebook instance itself. If you have a larger dataset that will not fit in a notebook instance memory, inspect the dataset offline using a big data analytics tool like Apache Spark. [Deequ](https://github.com/awslabs/deequ) is a library built on top of Apache Spark that can be helpful for performing checks on large datasets. Autopilot is capable of handling datasets up to 5 GB.


Read the data into a Pandas data frame and take a look.

In [8]:
import pandas as pd

data = pd.read_csv(local_data_path)
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 10)         # Keep the output on one page
data

,age,age_bins,job,marital,job_marital,education,default,housing,loan,cons.conf.idx,cons.price.idx,emp.var.rate,euribor3m,nr.employed,y,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
0,56,4,House maid,married,House maid-married,1,False,False,False,-36.4,93.994,1.1,4.857,5191.0,False,telephone,may,mon,261,1,999,0,nonexistent
1,35,2,Blue Collar,married,Blue Collar-married,2,False,True,False,-36.4,93.994,1.1,4.857,5191.0,False,telephone,may,mon,146,1,999,0,nonexistent
2,59,4,technician,married,technician-married,3,False,True,False,-36.4,93.994,1.1,4.857,5191.0,False,telephone,may,mon,386,1,999,0,nonexistent
3,34,2,services,married,services-married,4,False,False,False,-36.4,93.994,1.1,4.857,5191.0,False,telephone,may,mon,365,1,999,0,nonexistent
4,32,2,entrepreneur,married,entrepreneur-married,4,False,True,False,-36.4,93.994,1.1,4.857,5191.0,False,telephone,may,mon,314,1,999,0,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,64,4,retired,divorced,retired-divorced,5,False,True,False,-50.8,94.767,-1.1,1.028,4963.6,False,cellular,NaN,fri,151,3,999,0,nonexistent
41184,37,2,Admin,married,Admin-married,6,False,True,False,-50.8,94.767,-1.1,1.028,4963.6,True,cellular,NaN,fri,281,1,999,0,nonexistent
41185,73,5,retired,married,retired-married,5,False,True,False,-50.8,94.767,-1.1,1.028,4963.6,True,cellular,NaN,fri,334,1,999,0,nonexistent
41186,56,4,retired,married,retired-married,6,False,True,False,-50.8,94.767,-1.1,1.028,4963.6,False,cellular,NaN,fri,189,2,999,0,nonexistent


Note that there are 20 features to help predict the target column 'y'.

Amazon SageMaker Autopilot takes care of preprocessing your data for you. You do not need to perform conventional data preprocssing techniques such as handling missing values, converting categorical features to numeric features, scaling data, and handling more complicated data types.

Moreover, splitting the dataset into training and validation splits is not necessary. Autopilot takes care of this for you. You may, however, want to split out a test set. That's next, although you use it for batch inference at the end instead of testing the model.


### Reserve some data for calling batch inference on the model

Divide the data into training and testing splits. The training split is used by SageMaker Autopilot. The testing split is reserved to perform inference using the suggested model.


In [9]:
train_data = data.sample(frac=0.8,random_state=200)

test_data = data.drop(train_data.index)

test_data_no_target = test_data.drop(columns=['y'])

### Upload the dataset to Amazon S3
Copy the file to Amazon Simple Storage Service (Amazon S3) in a .csv format for Amazon SageMaker training to use.

In [10]:
train_file = 'train_data.csv';
train_data.to_csv(train_file, index=False, header=True)
train_data_s3_path = session.upload_data(path=train_file, key_prefix=prefix + "/train")
print('Train data uploaded to: ' + train_data_s3_path)

test_file = 'test_data.csv';
test_data_no_target.to_csv(test_file, index=False, header=False)
test_data_s3_path = session.upload_data(path=test_file, key_prefix=prefix + "/test")
print('Test data uploaded to: ' + test_data_s3_path)

Train data uploaded to: s3://sagemaker-us-west-2-222702392921/sagemaker/autopilot-dm/train/train_data.csv
Test data uploaded to: s3://sagemaker-us-west-2-222702392921/sagemaker/autopilot-dm/test/test_data.csv


## Setting up the SageMaker Autopilot Job<a name="Settingup"></a>

After uploading the dataset to Amazon S3, you can invoke Autopilot to find the best ML pipeline to train a model on this dataset. 

The required inputs for invoking a Autopilot job are:
* Amazon S3 location for input dataset and for all output artifacts
* Name of the column of the dataset you want to predict (`y` in this case) 
* An IAM role

Currently Autopilot supports only tabular datasets in CSV format. Either all files should have a header row, or the first file of the dataset, when sorted in alphabetical/lexical order, is expected to have a header row.

In [11]:
input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': 's3://{}/{}/train'.format(bucket,prefix)
        }
      },
      'TargetAttributeName': 'y'
    }
  ]

output_data_config = {
    'S3OutputPath': 's3://{}/{}/output'.format(bucket,prefix)
  }

You can also specify the type of problem you want to solve with your dataset (`Regression, MulticlassClassification, BinaryClassification`). In case you are not sure, SageMaker Autopilot will infer the problem type based on statistics of the target column (the column you want to predict). 

You have the option to limit the running time of a SageMaker Autopilot job by providing either the maximum number of pipeline evaluations or candidates (one pipeline evaluation is called a `Candidate` because it generates a candidate model) or providing the total time allocated for the overall Autopilot job. Under default settings, this job takes about four hours to run. This varies between runs because of the nature of the exploratory process Autopilot uses to find optimal training parameters.

## Launching the SageMaker Autopilot Job<a name="Launching"></a>

You can now launch the Autopilot job by calling the `create_auto_ml_job` API. 

In [15]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-banking-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      RoleArn=role)

AutoMLJobName: automl-banking-23-15-44-42


{'AutoMLJobArn': 'arn:aws:sagemaker:us-west-2:222702392921:automl-job/automl-banking-23-15-44-42',
 'ResponseMetadata': {'RequestId': 'bf47663b-9bac-4c0b-acbd-455d86a59ceb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'bf47663b-9bac-4c0b-acbd-455d86a59ceb',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '97',
   'date': 'Sun, 23 Feb 2020 15:44:42 GMT'},
  'RetryAttempts': 0}}

## Tracking SageMaker Autopilot job progress<a name="Tracking"></a>
SageMaker Autopilot job consists of the following high-level steps : 
* Analyzing Data, where the dataset is analyzed and Autopilot comes up with a list of ML pipelines that should be tried out on the dataset. The dataset is also split into train and validation sets.
* Feature Engineering, where Autopilot performs feature transformation on individual features of the dataset as well as at an aggregate level.
* Model Tuning, where the top performing pipeline is selected along with the optimal hyperparameters for the training algorithm (the last stage of the pipeline). 

In [16]:
print ('JobStatus - Secondary Status')
print('------------------------------')


describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
job_run_status = describe_response['AutoMLJobStatus']
    
while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response['AutoMLJobStatus']
    
    print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
    sleep(30)

JobStatus - Secondary Status
------------------------------
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InP

## Results

Now use the describe_auto_ml_job API to look up the best candidate selected by the SageMaker Autopilot job. 

In [17]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_name = best_candidate['CandidateName']
print(best_candidate)
print('\n')
print("CandidateName: " + best_candidate_name)
print("FinalAutoMLJobObjectiveMetricName: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("FinalAutoMLJobObjectiveMetricValue: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

{'CandidateName': 'tuning-job-1-6cc6bcf4b0194c409e-156-782e3a3e', 'FinalAutoMLJobObjectiveMetric': {'MetricName': 'validation:accuracy', 'Value': 0.9188410043716431}, 'ObjectiveStatus': 'Succeeded', 'CandidateSteps': [{'CandidateStepType': 'AWS::SageMaker::ProcessingJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-west-2:222702392921:processing-job/db-1-edf485d9bcb3413e8983e19a97261a10bad23e0670424fcc9114004c4c', 'CandidateStepName': 'db-1-edf485d9bcb3413e8983e19a97261a10bad23e0670424fcc9114004c4c'}, {'CandidateStepType': 'AWS::SageMaker::TrainingJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-west-2:222702392921:training-job/automl-ban-dpp6-1-f0d402351a7a48308aec845aa0fec7dd03055393c0834', 'CandidateStepName': 'automl-ban-dpp6-1-f0d402351a7a48308aec845aa0fec7dd03055393c0834'}, {'CandidateStepType': 'AWS::SageMaker::TransformJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-west-2:222702392921:transform-job/automl-ban-dpp6-rpb-1-f4ad81a34b5642f5bfd2b338f737b024ee845a841', 'CandidateStepNa

In [18]:
sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

{'AutoMLJobName': 'automl-banking-23-15-44-42',
 'AutoMLJobArn': 'arn:aws:sagemaker:us-west-2:222702392921:automl-job/automl-banking-23-15-44-42',
 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
     'S3Uri': 's3://sagemaker-us-west-2-222702392921/sagemaker/autopilot-dm/train'}},
   'TargetAttributeName': 'y'}],
 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-west-2-222702392921/sagemaker/autopilot-dm/output'},
 'RoleArn': 'arn:aws:iam::222702392921:role/trifacta_master',
 'CreationTime': datetime.datetime(2020, 2, 23, 7, 44, 42, 596000, tzinfo=tzlocal()),
 'EndTime': datetime.datetime(2020, 2, 23, 9, 32, 30, 895000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2020, 2, 23, 9, 32, 30, 933000, tzinfo=tzlocal()),
 'BestCandidate': {'CandidateName': 'tuning-job-1-6cc6bcf4b0194c409e-156-782e3a3e',
  'FinalAutoMLJobObjectiveMetric': {'MetricName': 'validation:accuracy',
   'Value': 0.9188410043716431},
  'ObjectiveStatus': 'Succeeded',
 

### Perform batch inference using the best candidate

Now that you have successfully completed the SageMaker Autopilot job on the dataset, create a model from any of the candidates by using [Inference Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html). 

In [19]:
model_name = 'automl-banking-model-' + timestamp_suffix

model = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)

print('Model ARN corresponding to the best candidate is : {}'.format(model['ModelArn']))

Model ARN corresponding to the best candidate is : arn:aws:sagemaker:us-west-2:222702392921:model/automl-banking-model-23-15-44-42


You can use batch inference by using Amazon SageMaker batch transform. The same model can also be deployed to perform online inference using Amazon SageMaker hosting.

In [20]:
transform_job_name = 'automl-banking-transform-' + timestamp_suffix

transform_input = {
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': test_data_s3_path
            }
        },
        'ContentType': 'text/csv',
        'CompressionType': 'None',
        'SplitType': 'Line'
    }

transform_output = {
        'S3OutputPath': 's3://{}/{}/inference-results'.format(bucket,prefix),
    }

transform_resources = {
        'InstanceType': 'ml.m5.4xlarge',
        'InstanceCount': 1
    }

sm.create_transform_job(TransformJobName = transform_job_name,
                        ModelName = model_name,
                        TransformInput = transform_input,
                        TransformOutput = transform_output,
                        TransformResources = transform_resources
)

{'TransformJobArn': 'arn:aws:sagemaker:us-west-2:222702392921:transform-job/automl-banking-transform-23-15-44-42',
 'ResponseMetadata': {'RequestId': '10d39b42-ac7b-4749-80ba-049e973bddc0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '10d39b42-ac7b-4749-80ba-049e973bddc0',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '113',
   'date': 'Sun, 23 Feb 2020 17:33:08 GMT'},
  'RetryAttempts': 0}}

Watch the transform job for completion.

In [21]:
print ('JobStatus')
print('----------')


describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
job_run_status = describe_response['TransformJobStatus']
print (job_run_status)

while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
    job_run_status = describe_response['TransformJobStatus']
    print (job_run_status)
    sleep(30)

JobStatus
----------
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
Completed


Now let's view the results of the transform job:

In [22]:
s3_output_key = '{}/inference-results/test_data.csv.out'.format(prefix);
local_inference_results_path = 'inference_results.csv'

s3 = boto3.resource('s3')
inference_results_bucket = s3.Bucket(session.default_bucket())

inference_results_bucket.download_file(s3_output_key, local_inference_results_path);

data = pd.read_csv(local_inference_results_path, sep=';')
pd.set_option('display.max_rows', 10)         # Keep the output on one page
data

,False
0,False
1,False
2,False
3,False
4,False
...,...
8232,False
8233,True
8234,False
8235,False


### View other candidates explored by SageMaker Autopilot
You can view all the candidates (pipeline evaluations with different hyperparameter combinations) that were explored by SageMaker Autopilot and sort them by their final performance metric.

In [23]:
candidates = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, SortBy='FinalObjectiveMetricValue')['Candidates']
index = 1
for candidate in candidates:
  print (str(index) + "  " + candidate['CandidateName'] + "  " + str(candidate['FinalAutoMLJobObjectiveMetric']['Value']))
  index += 1

1  tuning-job-1-6cc6bcf4b0194c409e-156-782e3a3e  0.9188410043716431
2  tuning-job-1-6cc6bcf4b0194c409e-229-2537a76e  0.9183859825134277
3  tuning-job-1-6cc6bcf4b0194c409e-091-195613c8  0.9182339906692505
4  tuning-job-1-6cc6bcf4b0194c409e-112-60eb4103  0.9177790284156799
5  tuning-job-1-6cc6bcf4b0194c409e-089-34a190fd  0.9174759984016418
6  tuning-job-1-6cc6bcf4b0194c409e-128-19d36064  0.9171720147132874
7  tuning-job-1-6cc6bcf4b0194c409e-133-3303a527  0.9170209765434265
8  tuning-job-1-6cc6bcf4b0194c409e-210-ad99d7d0  0.9168689846992493
9  tuning-job-1-6cc6bcf4b0194c409e-048-28ee58c3  0.916716992855072
10  tuning-job-1-6cc6bcf4b0194c409e-071-9e8ef83e  0.916566014289856


### Candidate Generation Notebook
    
Sagemaker AutoPilot also auto-generates a Candidate Definitions notebook. This notebook can be used to interactively step through the various steps taken by the Sagemaker Autopilot to arrive at the best candidate. This notebook can also be used to override various runtime parameters like parallelism, hardware used, algorithms explored, feature extraction scripts and more.
    
The notebook can be downloaded from the following Amazon S3 location:

In [24]:
sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['AutoMLJobArtifacts']['CandidateDefinitionNotebookLocation']


's3://sagemaker-us-west-2-222702392921/sagemaker/autopilot-dm/output/automl-banking-23-15-44-42/sagemaker-automl-candidates/pr-1-7dcbd48a1e0144f0befd36e97be8c0f1fe39d6ed46614078b0338c1709/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb'

### Data Exploration Notebook
Sagemaker Autopilot also auto-generates a Data Exploration notebook, which can be downloaded from the following Amazon S3 location:

In [25]:
sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['AutoMLJobArtifacts']['DataExplorationNotebookLocation']


's3://sagemaker-us-west-2-222702392921/sagemaker/autopilot-dm/output/automl-banking-23-15-44-42/sagemaker-automl-candidates/pr-1-7dcbd48a1e0144f0befd36e97be8c0f1fe39d6ed46614078b0338c1709/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb'

## Cleanup

The Autopilot job creates many underlying artifacts such as dataset splits, preprocessing scripts, or preprocessed data, etc. This code, when un-commented, deletes them. This operation deletes all the generated models and the auto-generated notebooks as well. 

In [26]:
#s3 = boto3.resource('s3')
#bucket = s3.Bucket(bucket)

#job_outputs_prefix = '{}/output/{}'.format(prefix,auto_ml_job_name)
#bucket.objects.filter(Prefix=job_outputs_prefix).delete()